In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import math
import seaborn as sns

In [42]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import auc,classification_report,accuracy_score,roc_curve

In [3]:
data = {'Soap': [ 4,4.5 ,  5,    5.5 ,  6,  6.5 ,  7  ],
        'Suds': [33 , 42    ,45,    51,  53 , 61,   62 ]
        }

df = pd.DataFrame(data, columns = ['Soap', 'Suds'])

df

,Soap,Suds
0,4.0,33
1,4.5,42
2,5.0,45
3,5.5,51
4,6.0,53
5,6.5,61
6,7.0,62


In [4]:
X_train, X_test, y_train, y_test = train_test_split(df['Soap'],df['Suds'] , random_state=42)

In [11]:
#Standardizing the X_train and X_test daatsets
mu = np.mean(X_train, 0)
sigma = np.std(X_train, 0)

X_train = (X_train - mu ) / sigma

#We use the same mean and SD as the one of X_train as we dont know the mean of X_test
X_test = (X_test - mu ) / sigma

#Standardizing the y_train data
mu_y = np.mean(y_train, 0)
sigma_y = np.std(y_train, 0, ddof = 0)

y_train = (y_train - mu_y ) / sigma_y

In [20]:
k_list = [x for x in range(1,50,1)]

# Calculating the distance matrix using numpy broadcasting technique 
distance = np.sqrt(((X_train[:] - X_test[:].T) ** 2).sum(0))

#Sorting each data points of the distance matrix to reduce computational effort 
sorted_distance = np.argsort(distance, axis = 0)

#The knn function takes in the sorted distance and returns the RMSE of the 
def knn(X_train,X_test,y_train,y_test,sorted_distance,k):
    y_pred = np.zeros(y_test.shape)
    for row in range(len(X_test)):
        
        #Transforming the y_train values to adjust the scale. 
        y_pred[row] = y_train[sorted_distance[:row][:k]].mean() * sigma_y + mu_y

    RMSE = np.sqrt(np.mean((y_test - y_pred)**2))
    return RMSE

#Storing the RMSE values in a list for each k value 
rmse_list = []
for i in k_list:
    rmse_list.append(knn(X_train,X_test,y_train,y_test,sorted_distance,i))

KeyError: "None of [Int64Index([0], dtype='int64')] are in the [index]"

In [18]:
X_test[:]

0   -2.828427
1   -2.121320
Name: Soap, dtype: float64

In [5]:
def knn(data, query, k, distance_fn, choice_fn):
    neighbor_distances_and_indices = []
    
    for index, example in enumerate(data):
        distance = distance_fn(example[:-1], query)
        neighbor_distances_and_indices.append((distance, index))
    sorted_neighbor_distances_and_indices = sorted(neighbor_distances_and_indices)
    k_nearest_distances_and_indices = sorted_neighbor_distances_and_indices[:k]
    k_nearest_labels = [data[i][-1] for distance, i in k_nearest_distances_and_indices]
    return k_nearest_distances_and_indices , choice_fn(k_nearest_labels)

In [6]:
def predict(data, queries, k, distance_fn, choice_fn):
    predictions=[]
    for query in queries:
        k_nearest_neighbors, prediction = knn(data, query, k, distance_fn, choice_fn)
        predictions.append(prediction)
    return predictions      


In [39]:
def euclidean_distance(point1, point2):
    sum_squared_distance = 0
    for i in range(len(point1)):
        sum_squared_distance += math.pow(point1[i] - point2[i], 2)
    return math.sqrt(sum_squared_distance)

In [8]:
def mode(labels):
    return Counter(labels).most_common(1)[0][0]

In [9]:
def mean(labels):
    return sum(labels) / len(labels)

In [35]:
x1 = np.array([4,4.5,5,5.5,6,6.5,7], dtype='float64')
y = np.array([33, 42, 45, 51, 53, 61, 62], dtype='float64')

In [36]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(x1,y,test_size=0.22)

In [41]:

x1 = np.array([4,4.5,5,5.5,6,6.5,7], dtype='float64')
y = np.array([33, 42, 45, 51, 53, 61, 62], dtype='float64')
data = np.stack((x1, y), axis=1)
inp = [[x] for x in x1]
for k in range(1,8):
    y_hat=predict(data, inp , k, distance_fn=euclidean_distance, choice_fn=mean)
    print(y_hat)

[33.0, 42.0, 45.0, 51.0, 53.0, 61.0, 62.0]
[37.5, 37.5, 43.5, 48.0, 52.0, 57.0, 61.5]
[40.0, 40.0, 46.0, 49.666666666666664, 55.0, 58.666666666666664, 58.666666666666664]
[42.75, 42.75, 42.75, 47.75, 52.5, 56.75, 56.75]
[44.8, 44.8, 44.8, 50.4, 54.4, 54.4, 54.4]
[47.5, 47.5, 47.5, 47.5, 52.333333333333336, 52.333333333333336, 52.333333333333336]
[49.57142857142857, 49.57142857142857, 49.57142857142857, 49.57142857142857, 49.57142857142857, 49.57142857142857, 49.57142857142857]


In [43]:
print ("KNN classifier")
print (classification_report(y,y_hat))

KNN classifier


ValueError: Classification metrics can't handle a mix of multiclass and continuous targets

In [47]:
for k in range(1,10):
    knn = KNeighborsClassifier(n_neighbors=k)
    model = knn.fit(x1,y)
    y_predict_proba = model.predict_proba(x1).tolist()
    probabilities = np.array(y_predict_proba)[:, 1]
    fpr, tpr, _ = roc_curve(Y_test, probabilities)
    roc_auc = auc(fpr, tpr)
    color = ['blue', 'orange', 'red', 'green', 'coral',
             'grey', 'indigo', 'gold', 'lime', 'olive',
             'pink', 'navy', 'magenta', 'yellow', 'tomato',
             'turquoise', 'yellowgreen', 'maroon', 'lightblue']
    plt.plot(fpr, tpr, color=color[k - 1], lw=1, label='ROC (area = %f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()

ValueError: Expected 2D array, got 1D array instead:
array=[4.  4.5 5.  5.5 6.  6.5 7. ].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.